In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
import os

import numpy as np
import pandas as pd
import sklearn
from tqdm.auto import tqdm

import scipy.stats
import sklearn.neighbors

from sklearn.model_selection import KFold, GridSearchCV

In [9]:
dataset_root_dir = 'dhs_train'

In [10]:
train_df = pd.read_csv("train.csv")


In [11]:
test_df = pd.read_csv("test_masked.csv")

In [12]:
test_df

,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,cluster_id,adm1fips,adm1dhs,urban,path
0,IA-2015-7-00010009,IA,2015,9.220903,92.781530,22.0,2.721812,10009,NaN,1,R,dhs_valid/IA-2015-7-00110896.npz
1,IA-2015-7-00010011,IA,2015,7.028410,93.883430,20.0,2.287279,10011,NaN,1,R,dhs_valid/IA-2015-7-00120669.npz
2,IA-2015-7-00010017,IA,2015,12.371448,92.783665,22.0,0.677109,10017,NaN,1,R,dhs_valid/IA-2015-7-00310941.npz
3,IA-2015-7-00010044,IA,2015,11.727304,92.719257,21.0,1.793683,10044,NaN,1,R,dhs_valid/IA-2015-7-00332819.npz
4,IA-2015-7-00010060,IA,2015,9.185310,92.777645,22.0,2.758168,10060,NaN,1,R,dhs_valid/IA-2015-7-00030310.npz
...,...,...,...,...,...,...,...,...,...,...,...,...
2670,IA-2015-7-00360403,IA,2015,17.410860,78.558810,19.0,2.823787,360403,NaN,36,U,dhs_valid/IA-2015-7-00100002.npz
2671,IA-2015-7-00360454,IA,2015,17.450098,78.360138,22.0,1.990244,360454,NaN,36,R,dhs_valid/IA-2015-7-00040683.npz
2672,IA-2015-7-00360474,IA,2015,17.973616,79.597408,20.0,2.633020,360474,NaN,36,U,dhs_valid/IA-2015-7-00200056.npz
2673,IA-2015-7-00360476,IA,2015,18.956406,77.934361,21.0,0.429279,360476,NaN,36,R,dhs_valid/IA-2015-7-00030367.npz


In [13]:
display(train_df.head())

,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,n_water,water_index,cluster_id,adm1fips,adm1dhs,urban,path
0,IA-2015-7-00010004,IA,2015,9.165413,92.742696,22.0,2.650768,22.0,5.000000,10004,NaN,1,R,dhs_train/IA-2015-7-00192065.npz
1,IA-2015-7-00010005,IA,2015,8.307356,93.093792,22.0,2.157784,22.0,5.000000,10005,NaN,1,R,dhs_train/IA-2015-7-00330893.npz
2,IA-2015-7-00010007,IA,2015,7.016968,93.893226,21.0,1.832751,21.0,4.619048,10007,NaN,1,R,dhs_train/IA-2015-7-00140183.npz
3,IA-2015-7-00010016,IA,2015,9.194938,92.800432,22.0,2.746096,22.0,5.000000,10016,NaN,1,R,dhs_train/IA-2015-7-00261454.npz
4,IA-2015-7-00010018,IA,2015,8.055606,93.543892,22.0,2.581869,22.0,5.000000,10018,NaN,1,R,dhs_train/IA-2015-7-00191303.npz


In [14]:
feature_cols = ["lat", "lon"]
label_cols = ['water_index']

In [15]:
feature_cols = ["lat", "lon"]
label_cols = ['water_index']
train_X = train_df[feature_cols]
train_Y = train_df[label_cols]
test_X = test_df[feature_cols]

In [16]:
import sys

!{sys.executable} -m pip install xgboost
from xgboost import XGBRegressor

model = XGBRegressor()

# Train the model on the training data
model.fit(train_X, train_Y)

# Use the trained model to predict the labels of the test data
test_Y_pred = model.predict(test_X)

In [17]:
test_Y_pred

solution_df = test_df.copy()

solution_df["water_index"] = test_Y_pred

filtered_solution_df = solution_df[["DHSID_EA", "water_index"]]
filtered_solution_df.to_csv("solution.csv", index=False)

In [18]:
train_X

,lat,lon
0,9.165413,92.742696
1,8.307356,93.093792
2,7.016968,93.893226
3,9.194938,92.800432
4,8.055606,93.543892
...,...,...
18717,17.184983,79.989186
18718,17.465781,78.418820
18719,17.939231,79.512509
18720,17.420175,78.491013


In [25]:
!pip3 install tensorflow==2.11.0
!pip3 install -q keras==2.4.3

  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
intel-tensorflow 2.9.1 requires flatbuffers<2,>=1.12, but you have flatbuffers 23.3.3 which is incompatible.
intel-tensorflow 2.9.1 requires keras<2.10.0,>=2.9.0rc0, but you have keras 2.11.0 which is incompatible.
intel-tensorflow 2.9.1 requires tensorboard<2.10,>=2.9, but you have tensorboard 2.11.2 which is incompatible.
intel-tensorflow 2.9.1 requires tensorflow-estimator<2.10.0,>=2.9.0rc0, but you have tensorflow-estimator 2.11.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.11.0 requires keras<2.12,>=2.11.0, but you have keras 2.4.3 which is incompatible.
intel-tensorflow 2.9.1 requires flatbuffers<2,>=1.12, but you have flatbuffers 23.3.3 

In [26]:
import tensorflow as tf

In [27]:
import sys

# !{sys.executable} -m pip install xgboost

import PIL 
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.applications import VGG16
from tensorflow.keras.utils import img_to_array, load_img

# from keras.applications import VGG16
# from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

# Load the data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_masked.csv')

# Define the pre-trained CNN model
model = VGG16(weights=None, include_top=False, input_shape=(255,255,8))

import matplotlib.pyplot as plt
from keras.applications.vgg16 import preprocess_input
from PIL import Image

plt2 = plt
# Define a function to extract features from an image
def extract_features(filepath):
    
    # print(filepath)
    # # Load the image data from the npz file
    # np.savez_compressed(filepath, 'arr_0')
    # # Load the image data from the npz file
    # image_data = np.load(filepath)
    
    # print(image_data)
    try:
        with np.load(filepath) as data:
            for key in data.keys():
                # print(key)                        
                # print(data[key].shape)  
                # print(data[key]) 
                # im = Image.fromarray(data[key])
                # im.save('./processed/' + key + '.png')
                image_data = data[key]
                image_data = np.moveaxis(image_data, 0, -1)
                image_data.shape
    except:
        print("Error")
        image_data = np.zeros((255, 255, 8))
    # Reshape the data to match the input shape of the model
    image_data = image_data.reshape((1, image_data.shape[0], image_data.shape[1], image_data.shape[2]))
    # Preprocess the image data for the model
    image_data = preprocess_input(image_data)
    # Use the model to extract features from the image
    features = model.predict(image_data)
    # Flatten the features to a 1-dimensional array
    features = features.reshape((features.shape[0], -1))
    return features

# Extract features from all images in the training data
train_image_features = []
for filepath in train_df['path']:
    features = extract_features(filepath)
    train_image_features.append(features)
train_image_features = np.array(train_image_features)

# Extract features from all images in the test data
test_image_features = []
for filepath in test_df['path']:
    features = extract_features(filepath)
    test_image_features.append(features)
test_image_features = np.array(test_image_features)

# Combine the image features with the non-image features in the training data
train_X = np.concatenate([train_df[feature_cols].values, train_image_features], axis=1)
train_Y = train_df[label_cols]

# Combine the image features with the non-image features in the test data
test_X = np.concatenate([test_df[feature_cols].values, test_image_features], axis=1)

# Train a model on the combined feature vectors
model.fit(train_X, train_Y)

# Use the trained model to predict the labels of the test data
test_Y_pred = model.predict(test_X)

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [28]:
with np.load('dhs_train/IA-2015-7-00360465.npz') as data:
        for key in data.keys():
            print(key)                        
            print(data[key].shape)  
            # print(data[key]) 
            im = Image.fromarray(data[key][3,:,:])
            plt.imshow(data[key][3,:,:])
            if im.mode != 'RGB':
                im = im.convert('RGB')
            im.save('./processed/' + 'aryan' + '.png')

x
(8, 255, 255)


NameError: name 'Image' is not defined

In [ ]:
train_df.head()

,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,n_water,water_index,cluster_id,adm1fips,adm1dhs,urban,path
0,IA-2015-7-00010004,IA,2015,9.165413,92.742696,22.0,2.650768,22.0,5.000000,10004,NaN,1,R,dhs_train/IA-2015-7-00192065.npz
1,IA-2015-7-00010005,IA,2015,8.307356,93.093792,22.0,2.157784,22.0,5.000000,10005,NaN,1,R,dhs_train/IA-2015-7-00330893.npz
2,IA-2015-7-00010007,IA,2015,7.016968,93.893226,21.0,1.832751,21.0,4.619048,10007,NaN,1,R,dhs_train/IA-2015-7-00140183.npz
3,IA-2015-7-00010016,IA,2015,9.194938,92.800432,22.0,2.746096,22.0,5.000000,10016,NaN,1,R,dhs_train/IA-2015-7-00261454.npz
4,IA-2015-7-00010018,IA,2015,8.055606,93.543892,22.0,2.581869,22.0,5.000000,10018,NaN,1,R,dhs_train/IA-2015-7-00191303.npz


In [ ]:
from sklearn.datasets import load_boston
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor, StackingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


feature_cols = ["lat", "lon", "n_asset" , "asset_index", "urban", 'cluster_id', 'adm1dhs']
label_cols = ['water_index']
train_X = train_df[feature_cols]
train_Y = train_df[label_cols]
test_X = test_df[feature_cols]

# change all entries in the urban column from R to 1 and U to 0
train_X['urban'] = train_X['urban'].map({'R': 1, 'U': 0})
test_X['urban'] = test_X['urban'].map({'R': 1, 'U': 0})


# Define the base models to use in the ensemble
base_models = [
    # LinearRegression(),
    # BaggingRegressor(n_estimators=50, random_state=42),
    RandomForestRegressor(n_estimators=50, random_state=42),
    GradientBoostingRegressor(n_estimators=50, random_state=42),
    AdaBoostRegressor(n_estimators=50, random_state=42),
    ExtraTreesRegressor(n_estimators=50, random_state=42),
    # VotingRegressor(estimators=[('rf', RandomForestRegressor(n_estimators=50, random_state=42)), ('gb', GradientBoostingRegressor(n_estimators=50, random_state=42)), ('ab', AdaBoostRegressor(n_estimators=50, random_state=42)), ('et', ExtraTreesRegressor(n_estimators=50, random_state=42))])
    

]

# Train each base model on the training data and make predictions on the testing data
base_model_predictions = []
for model in base_models:
    model.fit(train_X, train_Y)
    base_model_predictions.append(model.predict(test_X))
    


# Combine the predictions of the base models by taking their average
# ensemble_predictions = sum(base_model_predictions) / len(base_model_predictions)


C:\Users\barya\AppData\Local\Temp\ipykernel_32744\2464559816.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_X['urban'] = train_X['urban'].map({'R': 1, 'U': 0})
C:\Users\barya\AppData\Local\Temp\ipykernel_32744\2464559816.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_X['urban'] = test_X['urban'].map({'R': 1, 'U': 0})
C:\Users\barya\AppData\Local\Temp\ipykernel_32744\2464559816.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sh

In [ ]:
train_X

,lat,lon,n_asset,asset_index,urban,cluster_id,adm1dhs
0,9.165413,92.742696,22.0,2.650768,1,10004,1
1,8.307356,93.093792,22.0,2.157784,1,10005,1
2,7.016968,93.893226,21.0,1.832751,1,10007,1
3,9.194938,92.800432,22.0,2.746096,1,10016,1
4,8.055606,93.543892,22.0,2.581869,1,10018,1
...,...,...,...,...,...,...,...
18717,17.184983,79.989186,22.0,0.629311,1,360473,36
18718,17.465781,78.418820,20.0,2.704372,0,360475,36
18719,17.939231,79.512509,17.0,1.315361,1,360480,36
18720,17.420175,78.491013,23.0,2.920893,0,360481,36


In [ ]:
# get the mean of all the predictions in one array 
ensemble_predictions = np.zeros(len(base_model_predictions[0]))
for i in range(len(base_model_predictions)):
    if i == 0:
        ensemble_predictions = base_model_predictions[i]
    else:
        ensemble_predictions += base_model_predictions[i]

ensemble_predictions = ensemble_predictions / len(base_model_predictions)





In [ ]:
base_model_predictions

[array([19.77668811, 19.061449  , 17.55640745, ..., 19.31940015,
        17.290865  , 17.08592288]),
 array([4.95061687, 4.73452243, 4.50296768, ..., 4.9019302 , 4.27769927,
        4.53488461]),
 array([4.82788942, 4.50118547, 4.24971683, ..., 4.70892402, 4.03881356,
        4.24971683]),
 array([5.        , 4.92996864, 4.38597403, ..., 4.84293106, 4.42484279,
        4.11468267])]

In [ ]:
ensemble_predictions

array([4.94417203, 4.76536225, 4.38910186, ..., 4.82985004, 4.32271625,
       4.27148072])

In [ ]:
solution_df = test_df.copy()

solution_df["water_index"] = ensemble_predictions

filtered_solution_df = solution_df[["DHSID_EA", "water_index"]]
filtered_solution_df.to_csv("solution.csv", index=False)

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the DeepSDG model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(8, 255, 255)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)


Using TensorFlow backend.


ImportError: cannot import name 'transpose_shape' from 'keras.utils.generic_utils' (c:\Users\barya\.conda\envs\posenet\lib\site-packages\keras\utils\generic_utils.py)

In [14]:
import numpy as np
import pandas as pd

train_dataset = pd.read_csv('train.csv')


# select 500 random samples from the training dataset
train_subset = train_dataset.sample(n=1000, random_state=42)

# select 100 random samples from the training dataset for validation
# test_dataset = train_dataset.sample(n=100, random_state=42)

validation_dataset = train_dataset.sample(n=100, random_state=42)

training_images = []
for filepath in train_subset['path']:
    try:
        with np.load(filepath) as data:
            for key in data.keys():
                image_data = data[key]
    except:
        print("Error")
        image_data = np.zeros((255, 255, 8))
    training_images.append(image_data)

training_scores = train_subset['water_index'].values

# testing_images = []
# for filepath in test_dataset['path']:
#     try:
#         with np.load(filepath) as data:
#             for key in data.keys():
#                 image_data = data[key]
#     except:
#         print("Error")
#         image_data = np.zeros((255, 255, 8))
#     testing_images.append(image_data)

# testing_scores = test_dataset['water_index'].values


validation_images = []
for filepath in validation_dataset['path']:
    try:
        with np.load(filepath) as data:
            for key in data.keys():
                image_data = data[key]
    except:
        print("Error")
        image_data = np.zeros((255, 255, 8))
    validation_images.append(image_data)

In [18]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.models import Model
from keras.backend import set_floatx

set_floatx('float32')
# tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)


# Define the input shape
input_shape = (8, 255, 255)

# Define the input layer
inputs = Input(shape=input_shape)

# Define the convolutional layers
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(pool2)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

# Flatten the output of the convolutional layers
flatten = Flatten()(pool3)

# Define the fully connected layers
fc1 = Dense(512, activation='relu')(flatten)
dropout1 = Dropout(0.5)(fc1)
fc2 = Dense(256, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(fc2)

# Define the output layer
output = Dense(1, activation='linear')(dropout2)

# Define the model
model = Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

# Train the model
with tf.device('/GPU:0'):
    model.fit(np.array(training_images), np.array(training_scores), epochs=20, batch_size=16, validation_split=0.2)

# Generate some dummy data for prediction
x_pred = np.array(validation_images)


# Use the model to make predictions
with tf.device('/GPU:0'):
    y_pred = model.predict(x_pred)

print(y_pred)


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [13]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11676952372607146469
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1736599143
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11824712242411943943
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]